In [1859]:

#Code block given as a starting point for project 1: 

import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# 7 Assests (Nvidia, Advanced Micro Devices, Gamestop, Walt Disney Co, Pfizer inc, Exxon Mobil Corp, Target Corp)
tickers =['NVDA','AMD','GME','DIS','PFE','XOM','TGT']

# Gathers asset's data from past 5 years (1826 days) to today 
start_date = dt.datetime.today() - pd.Timedelta(1826, "d")
end_date = dt.datetime.today()

#Downloads stock data
portfolio =yf.download (start=start_date,end=end_date, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])
portfolio.head()


/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[**************        29%%                      ]  2 of 7 completed/Users/jeremyp/Desktop/pandas_proje

Price       Adj Close                                                         \
Ticker            AMD         DIS       GME       NVDA        PFE        TGT   
Date                                                                           
2019-02-28  23.530001  111.115471  2.829015  38.304882  33.759628  64.661705   
2019-03-01  23.680000  112.267601  2.841105  38.848686  33.767418  64.928741   
2019-03-04  23.370001  112.582703  2.797582  38.930630  33.541576  64.688408   
2019-03-05  23.500000  112.257751  2.804836  38.866070  33.401402  67.652664   
2019-03-06  22.410000  113.094749  2.795164  37.756107  32.599262  68.453812   

Price                      Close                      ...       Open  \
Ticker            XOM        AMD         DIS     GME  ...        PFE   
Date                                                  ...              
2019-02-28  61.304325  23.530001  112.839996  2.9250  ...  40.749527   
2019-03-01  62.056763  23.680000  114.010002  2.9375  ...  41.375713   
2019-03-04  62.297234  23.370001  114.330002  2.8925  ...  41.214420   
2019-03-05  62.204170  23.500000  114.000000  2.9000  ...  40.882355   
2019-03-06  61.498253  22.410000  114.849998  2.8900  ...  40.626186   

Price                               Volume                               \
Ticker            TGT        XOM       AMD      DIS       GME      NVDA   
Date                                                                      
2019-02-28  73.120003  79.449997  39384900  6716700  11280800  32494400   
2019-03-01  73.589996  79.379997  48084000  6996900   7604800  35718800   
2019-03-04  73.449997  80.239998  48147700  6015000   8914400  40999200   
2019-03-05  76.489998  80.120003  35462600  5225100  22151600  36292400   
2019-03-06  76.820000  78.849998  60479400  7352800  13251600  40352400   

Price                                     
Ticker           PFE       TGT       XOM  
Date                                      
2019-02-28  39002427   5375600  14597900  
2019-03-01  26667781   6833800  15419400  
2019-03-04  22805820   7383800  18089900  
2019-03-05  26529812  17015100  13745500  
2019-03-06  22410359   7874200  18633600  

[5 rows x 42 columns]

In [1860]:
portfolio = portfolio ['Adj Close'] #accounts for splits 
portfolio.head()

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2019-02-28,23.530001,111.115471,2.829015,38.304882,33.759628,64.661705,61.304325
2019-03-01,23.680000,112.267601,2.841105,38.848686,33.767418,64.928741,62.056763
2019-03-04,23.370001,112.582703,2.797582,38.930630,33.541576,64.688408,62.297234
2019-03-05,23.500000,112.257751,2.804836,38.866070,33.401402,67.652664,62.204170
2019-03-06,22.410000,113.094749,2.795164,37.756107,32.599262,68.453812,61.498253


In [1861]:

daily_returns = portfolio.pct_change()
daily_returns= daily_returns.drop(index = daily_returns.index[0], axis=0, inplace=False) # drops row with NaN values: row of 2019-02-27 because the numerator = 0 when calculating daily returns since theres no data from previous day to calculate for the 27th.
print(daily_returns)

Ticker           AMD       DIS       GME      NVDA       PFE       TGT  \
Date                                                                     
2019-03-01  0.006375  0.010369  0.004273  0.014197  0.000231  0.004130   
2019-03-04 -0.013091  0.002807 -0.015319  0.002109 -0.006688 -0.003701   
2019-03-05  0.005563 -0.002886  0.002593 -0.001658 -0.004179  0.045824   
2019-03-06 -0.046383  0.007456 -0.003448 -0.028559 -0.024015  0.011842   
2019-03-07 -0.014726 -0.007314  0.002595 -0.018349 -0.011227 -0.009493   
...              ...       ...       ...       ...       ...       ...   
2024-02-22  0.106945 -0.000279 -0.003729  0.164009 -0.004337  0.010417   
2024-02-23 -0.029363  0.000929  0.011228  0.003552  0.007623  0.007583   
2024-02-26 -0.002889 -0.000557  0.012583  0.003489 -0.020893 -0.008780   
2024-02-27  0.011306  0.016159  0.038743 -0.004944 -0.010670  0.012254   
2024-02-28 -0.008202  0.012612  0.002111 -0.013189  0.005578 -0.003619   

Ticker           XOM  
Date          

LECTURE NOTES:
For returns, use Adjusted Close instead of Close to normalize everything to it's current price point.
%change is a function for Rolling
To calculate return : Assestname*(today's price)-Assetnames*(yesterday's price)/Assetnames*(yesterday's price)
This gives the mean daily returns based on Adj Close price. 
The NaN values do not compute a % change for row 2019-02-26 because there are no values for 2019-02-25 to roll onto the next day.
The following rows are the % change from the 26 to the 27, then the 27 to 28 and so on. Rolling period defualt is one day
Using portfolio.pct_change().mean() calculates the mean for each column. 
Dont use 252 days to calculate annual returns (252 trading days within a year excluding the weekends). Why?

In [1892]:
# Column 3: Anualized volatility: trailing 3 months
#Source: https://www.youtube.com/watch?v=j4c2XqiJzRU

#Assuming average trading days per month = 21. 21 days * 3 = 63 days for 3-month period
days = 63
# Calculates annualized volatility trailing over 3-months 
volatility_three = daily_returns.rolling(window=days).std() * np.sqrt(days)*100

# Drop NaN values from the trailing window
volatility_three = volatility_three.dropna()
volatility_three

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2019-05-30,27.034975,14.113491,17.268109,18.286438,8.225602,14.195975,7.802077
2019-05-31,27.152352,14.096758,17.239890,18.383668,8.271414,14.194628,7.828957
2019-06-03,27.105503,14.096935,17.235727,18.407944,8.308179,14.479451,8.001807
2019-06-04,27.968923,14.167747,18.014836,19.735470,8.332930,14.186259,8.392799
2019-06-05,27.501706,14.170835,39.364361,19.570417,7.988768,14.166526,8.360452
...,...,...,...,...,...,...,...
2024-02-22,25.344263,15.391530,41.392981,23.405366,13.496725,10.238403,9.669576
2024-02-23,25.467569,15.368985,41.344844,23.347554,13.473429,10.231138,9.711629
2024-02-26,25.391912,15.365505,41.289196,23.121047,13.617927,10.282655,9.708424


In [1865]:
#Beta against an asset= the covariance(daily returns of assest)/variance(daily returns of what is being compared to: SPY,IWM or DIA)
#Beta = how a stock varies compared to current market
    #A measure of risk: 
    #   B= 1, volatile as market.
    #   B>1, more volatile. 
    #   B<1, less volatile. 

source for Columns 4-6: https://www.youtube.com/watch?v=_8DZxoEwpwE

In [1866]:
#Column4: Beta against SPY
assets = ['SPY','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

start_date2 = dt.datetime.today() - pd.Timedelta(365, "d")
end_date2 = dt.datetime.today()

ticker2 = yf.download(assets,start=start_date2, end=end_date2)
ticker2=ticker2['Adj Close']
ticker2.head()


/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[************          25%%                      ]  2

Ticker,AMD,DIS,GME,NVDA,PFE,SPY,TGT,XOM
Date,,,,,,,,
2023-02-28,78.580002,99.286415,19.230000,232.057053,38.584778,390.327576,163.035339,106.112984
2023-03-01,78.290001,98.219894,18.150000,226.879333,38.213867,388.830292,157.133163,107.059128
2023-03-02,80.440002,98.598656,18.190001,233.036591,38.632336,391.854309,160.558380,107.551506
2023-03-03,81.519997,100.811447,18.700001,238.794037,39.136406,398.138824,160.616425,108.912796
2023-03-06,81.160004,100.333008,18.650000,235.435516,39.098362,398.414612,159.842361,109.878250


In [1867]:
data= pd.DataFrame(ticker2)
data.head()

Ticker,AMD,DIS,GME,NVDA,PFE,SPY,TGT,XOM
Date,,,,,,,,
2023-02-28,78.580002,99.286415,19.230000,232.057053,38.584778,390.327576,163.035339,106.112984
2023-03-01,78.290001,98.219894,18.150000,226.879333,38.213867,388.830292,157.133163,107.059128
2023-03-02,80.440002,98.598656,18.190001,233.036591,38.632336,391.854309,160.558380,107.551506
2023-03-03,81.519997,100.811447,18.700001,238.794037,39.136406,398.138824,160.616425,108.912796
2023-03-06,81.160004,100.333008,18.650000,235.435516,39.098362,398.414612,159.842361,109.878250


In [1868]:
#Calculate daily returns 
daily_returns2 = ticker2.pct_change()
print(daily_returns2)


Ticker           AMD       DIS       GME      NVDA       PFE       SPY  \
Date                                                                     
2023-02-28       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-01 -0.003691 -0.010742 -0.056162 -0.022312 -0.009613 -0.003836   
2023-03-02  0.027462  0.003856  0.002204  0.027139  0.010951  0.007777   
2023-03-03  0.013426  0.022442  0.028037  0.024706  0.013048  0.016038   
2023-03-06 -0.004416 -0.004746 -0.002674 -0.014065 -0.000972  0.000693   
...              ...       ...       ...       ...       ...       ...   
2024-02-22  0.106945 -0.000279 -0.003729  0.164009 -0.004337  0.020695   
2024-02-23 -0.029363  0.000929  0.011228  0.003552  0.007623  0.000690   
2024-02-26 -0.002889 -0.000557  0.012583  0.003489 -0.020893 -0.003663   
2024-02-27  0.011306  0.016159  0.038743 -0.004944 -0.010670  0.001858   
2024-02-28 -0.008202  0.012612  0.002111 -0.013189  0.005578 -0.001322   

Ticker           TGT       XOM  
Date

In [1893]:
#Covariances and Variance
covariance= daily_returns2.cov()
variance= np.var(daily_returns2['SPY'])
beta =[] 

for i in assets:
    asset_beta = covariance.loc[i,'SPY']/variance
    beta.append(asset_beta)

beta

[1.0039999999999993,
 2.0592247604641893,
 1.9440126284539714,
 1.266943699977179,
 0.9662242052129884,
 0.4295140403080975,
 0.5172266111909422,
 0.8539094080053801]

In [1894]:
df=pd.DataFrame({'Asset': assets, 'Beta against SPY':beta})
df=df.round(2)
df

,Asset,Beta against SPY
0,SPY,1.00
1,NVDA,2.06
2,AMD,1.94
3,GME,1.27
4,DIS,0.97
5,PFE,0.43
6,XOM,0.52
7,TGT,0.85


In [1895]:
#Column 5: Beta against IWM

assets2 = ['IWM','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

#start_date2 = dt.datetime.today() - pd.Timedelta(365, "d")
#end_date2 = dt.datetime.today()

ticker3 = yf.download(assets2,start=start_date2, end=end_date2)
ticker3=ticker3['Adj Close'] #daily price
ticker3.head()


/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[************          25%%                      ]  2

Ticker,AMD,DIS,GME,IWM,NVDA,PFE,TGT,XOM
Date,,,,,,,,
2023-02-28,78.580002,99.286415,19.230000,185.417175,232.057053,38.584782,163.035339,106.112976
2023-03-01,78.290001,98.219894,18.150000,185.653671,226.879333,38.213871,157.133163,107.059128
2023-03-02,80.440002,98.598656,18.190001,186.097046,233.036606,38.632336,160.558365,107.551506
2023-03-03,81.519997,100.811447,18.700001,188.688431,238.794037,39.136402,160.616425,108.912788
2023-03-06,81.160004,100.333008,18.650000,185.929550,235.435532,39.098358,159.842377,109.878242


In [1872]:
data2= pd.DataFrame(ticker3)
data2.head()

daily_returns3 = ticker3.pct_change()
print(daily_returns3)


Ticker           AMD       DIS       GME       IWM      NVDA       PFE  \
Date                                                                     
2023-02-28       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-01 -0.003691 -0.010742 -0.056162  0.001276 -0.022312 -0.009613   
2023-03-02  0.027462  0.003856  0.002204  0.002388  0.027139  0.010951   
2023-03-03  0.013426  0.022442  0.028037  0.013925  0.024706  0.013048   
2023-03-06 -0.004416 -0.004746 -0.002674 -0.014621 -0.014065 -0.000972   
...              ...       ...       ...       ...       ...       ...   
2024-02-22  0.106945 -0.000279 -0.003729  0.007578  0.164009 -0.004337   
2024-02-23 -0.029363  0.000929  0.011228  0.002256  0.003552  0.007623   
2024-02-26 -0.002889 -0.000557  0.012583  0.006303  0.003489 -0.020893   
2024-02-27  0.011306  0.016159  0.038743  0.014169 -0.004944 -0.010670   
2024-02-28 -0.008202  0.012612  0.002111 -0.008235 -0.013189  0.005578   

Ticker           TGT       XOM  
Date

In [1873]:
covariance2= daily_returns3.cov()
variance2= np.var(daily_returns3['IWM'])
beta2 =[] 

for i in assets2:
    asset_beta2 = covariance2.loc[i,'IWM']/variance2
    beta2.append(asset_beta2)
print(beta2)

[1.004, 0.503969573931583, 0.5864234109929116, 1.1869076533141258, 0.5856481268768335, 0.22970618289688727, 0.44675588436701896, 0.6083764529688955]


In [1874]:
df2=pd.DataFrame({'Asset': assets2, 'Beta against IWM':beta2})
df2=df2.round(2)
print(df2)

  Asset  Beta against IWM
0   IWM              1.00
1  NVDA              0.50
2   AMD              0.59
3   GME              1.19
4   DIS              0.59
5   PFE              0.23
6   XOM              0.45
7   TGT              0.61


In [1875]:
#Column 6: Beta against DIA

assets3 = ['DIA','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

ticker4 = yf.download(assets3,start=start_date2, end=end_date2)
ticker4=ticker4['Adj Close'] #daily price
ticker4.head()

data3= pd.DataFrame(ticker4)
data3.head()


/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[************          25%%                      ]  2 of 8 completed/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_err

Ticker,AMD,DIA,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,,
2023-02-28,78.580002,320.276062,99.286415,19.230000,232.057053,38.584782,163.035355,106.112991
2023-03-01,78.290001,320.697815,98.219894,18.150000,226.879333,38.213871,157.133163,107.059128
2023-03-02,80.440002,324.071472,98.598656,18.190001,233.036591,38.632336,160.558380,107.551506
2023-03-03,81.519997,327.768799,100.811447,18.700001,238.794037,39.136398,160.616440,108.912788
2023-03-06,81.160004,328.229797,100.333008,18.650000,235.435516,39.098358,159.842361,109.878250


In [1876]:
daily_returns4 = ticker4.pct_change()
print(daily_returns4)

covariance3= daily_returns4.cov()
variance3= np.var(daily_returns4['DIA'])
beta3 =[] 

for i in assets3:
    asset_beta3 = covariance3.loc[i,'DIA']/variance3
    beta3.append(asset_beta3)
print(beta3)

Ticker           AMD       DIA       DIS       GME      NVDA       PFE  \
Date                                                                     
2023-02-28       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-01 -0.003691  0.001317 -0.010742 -0.056162 -0.022312 -0.009613   
2023-03-02  0.027462  0.010520  0.003856  0.002204  0.027139  0.010951   
2023-03-03  0.013426  0.011409  0.022442  0.028037  0.024706  0.013048   
2023-03-06 -0.004416  0.001406 -0.004746 -0.002674 -0.014065 -0.000972   
...              ...       ...       ...       ...       ...       ...   
2024-02-22  0.106945  0.011631 -0.000279 -0.003729  0.164009 -0.004337   
2024-02-23 -0.029363  0.001895  0.000929  0.011228  0.003552  0.007623   
2024-02-26 -0.002889 -0.001457 -0.000557  0.012583  0.003489 -0.020893   
2024-02-27  0.011306 -0.002687  0.016159  0.038743 -0.004944 -0.010670   
2024-02-28 -0.008202 -0.000154  0.012612  0.002111 -0.013189  0.005578   

Ticker           TGT       XOM  
Date

In [1877]:
df3=pd.DataFrame({'Asset': assets3, 'Beta against DIA':beta3})
df3=df3.round(2)
df3

,Asset,Beta against DIA
0,DIA,1.00
1,NVDA,1.23
2,AMD,1.20
3,GME,1.32
4,DIS,1.22
5,PFE,0.62
6,XOM,0.97
7,TGT,1.23


In [1878]:
#Column 7: Average Weekly drawdown 
#   ([52_weeks_Low - 52_weeks_High])/(52_weeks_High)
tickers =['NVDA','AMD','GME','DIS','PFE','XOM','TGT']

start_date3 = dt.datetime.today() - pd.Timedelta(365, "d")
end_date3 = dt.datetime.today()

portfolio2 =yf.download(start=start_date3,end =end_date3, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])['Adj Close']
portfolio2.head()

/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[**************        29%%                      ]  2

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-02-28,78.580002,99.286415,19.230000,232.057053,38.584782,163.035339,106.112976
2023-03-01,78.290001,98.219894,18.150000,226.879333,38.213867,157.133163,107.059128
2023-03-02,80.440002,98.598656,18.190001,233.036591,38.632336,160.558380,107.551506
2023-03-03,81.519997,100.811447,18.700001,238.794037,39.136402,160.616440,108.912788
2023-03-06,81.160004,100.333008,18.650000,235.435516,39.098358,159.842361,109.878242


In [1879]:
returns = portfolio2.pct_change() #daily returns
returns= returns.dropna()

#Changes frequency from daily to weekly returns
weekly_returns = returns.resample('W-Fri').last()

print("Weekly Returns: ")
weekly_returns.head()

Weekly Returns: 


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-03-03,0.013426,0.022442,0.028037,0.024706,0.013048,0.000362,0.012657
2023-03-10,-0.016185,-0.026732,-0.015411,-0.020097,-0.001774,-0.015314,-0.012370
2023-03-17,0.012836,-0.011560,-0.009547,0.007204,-0.006688,-0.022631,-0.011779
2023-03-24,-0.023235,-0.018262,0.062002,-0.015152,0.005227,0.000641,0.001160
2023-03-31,0.001328,0.020693,0.023111,0.014388,0.010401,0.033444,0.001553


In [1880]:
#2 Calculate Cumulative Returns
cumulative_max= weekly_returns.cummax()
#3 weekly drawdowns for 1y period=52 weeks
weekly_drawdown= (weekly_returns-cumulative_max)/ cumulative_max
weekly_drawdown.head()

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-03-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-10,-2.205467,-2.191132,-1.549658,-1.813454,-1.135964,-43.348378,-1.977360
2023-03-17,-0.043919,-1.515102,-1.340493,-0.708407,-1.512584,-63.584049,-1.930609
2023-03-24,-2.730584,-1.813706,0.000000,-1.613290,-0.599431,0.000000,-0.908325
2023-03-31,-0.901073,-0.077943,-0.627250,-0.417619,-0.202831,0.000000,-0.877331


In [1881]:
# This loop gives the avg for each weekly drawndown column
avg_drawdown=[]

#for i in weekly_drawdown:
 #   avg_drawdown= weekly_drawdown['AMD'].mean(),weekly_drawdown['DIS'].mean(),weekly_drawdown['GME'].mean(),weekly_drawdown['NVDA'].mean(),weekly_drawdown['PFE'].mean(),weekly_drawdown['TGT'].mean(),weekly_drawdown['XOM'].mean()

for i in tickers:
    avg_drawdown= weekly_drawdown[tickers].mean()  
avg_drawdown=avg_drawdown.round(4)

avg_drawdown


Ticker
NVDA   -0.9132
AMD    -0.9432
GME    -0.9907
DIS    -0.9794
PFE    -0.8989
XOM    -0.9751
TGT    -2.9880
dtype: float64

In [1882]:
#Column 8: Maximum Weekly Drawdown 
#Source: https://quant.stackexchange.com/questions/18094/how-can-i-calculate-the-maximum-drawdown-mdd-in-python
#        https://medium.com/cloudcraftz/measuring-maximum-drawdown-and-its-python-implementation-99a3963e158f

weekly_returns.head()
window=365
Roll_Max = weekly_returns.rolling(window, min_periods=1).max()
max_weekly_drawdown = weekly_returns/Roll_Max - 1.0

max_drawdown_values = max_weekly_drawdown.min()
max_drawdown_values=max_drawdown_values.round(4)

max_drawdown_values


Ticker
AMD     -2.7306
DIS     -2.1911
GME     -2.0075
NVDA    -1.9092
PFE     -2.5212
TGT    -63.5840
XOM     -2.3832
dtype: float64

In [1883]:
#Column 9: Total Return trailing 10 years

start_date = dt.datetime.today() - pd.Timedelta(3650, "d")
end_date = dt.datetime.today()

#Downloads stock data
portfolio3 =yf.download (start=start_date,end=end_date, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])['Adj Close']
portfolio3.inplace=True

portfolio3.tail()


/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]/Users/jeremyp/Desktop/pandas_project/ccny/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[**************        29%%                      ]  2

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2024-02-22,181.860001,107.639999,13.36,785.380005,27.549999,150.339996,104.760002
2024-02-23,176.520004,107.739998,13.51,788.169983,27.760000,151.479996,103.839996
2024-02-26,176.009995,107.680000,13.68,790.919983,27.180000,150.149994,104.250000
2024-02-27,178.000000,109.419998,14.21,787.010010,26.889999,151.990005,104.029999
2024-02-28,176.539993,110.800003,14.24,776.630005,27.040001,151.440002,104.320000


In [1884]:

current_price= (portfolio3.iloc[-1])
current_price

Ticker
AMD     176.539993
DIS     110.800003
GME      14.240000
NVDA    776.630005
PFE      27.040001
TGT     151.440002
XOM     104.320000
Name: 2024-02-28 00:00:00, dtype: float64

In [1885]:
portfolio3.head()


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2014-03-03,3.67,72.890755,6.697840,4.333338,20.830904,46.635635,61.979229
2014-03-04,3.70,74.954765,6.726656,4.378354,21.293373,46.161469,62.641178
2014-03-05,3.71,75.844566,6.978793,4.416262,21.332455,45.612007,60.875916
2014-03-06,3.73,76.449989,6.967987,4.354661,21.143551,45.717384,60.849949
2014-03-07,3.95,75.413406,6.937372,4.349923,21.124014,45.724907,61.648232


In [1886]:
starting_price= portfolio3.iloc[0]
starting_price

Ticker
AMD      3.670000
DIS     72.890755
GME      6.697840
NVDA     4.333338
PFE     20.830904
TGT     46.635635
XOM     61.979229
Name: 2014-03-03 00:00:00, dtype: float64

In [1887]:
for i in tickers:
    total_return=(((current_price-starting_price)/starting_price)*100).round(2)
    
total_return

Ticker
AMD      4710.35
DIS        52.01
GME       112.61
NVDA    17822.21
PFE        29.81
TGT       224.73
XOM        68.31
dtype: float64

In [1888]:
#Column 10 : Annualized total return trailing 10 years
annual_return = (((1+total_return)**(365/3650)-1)*100).round(2)
annual_return



Ticker
AMD     132.98
DIS      48.74
GME      60.52
NVDA    166.13
PFE      40.89
TGT      71.93
XOM      52.78
dtype: float64

In [1889]:
#Table 1

analysis = pd.DataFrame(index=tickers)

#Portfolio Wieght (must be equally weighted!) = 1/7 (since there are seven assests)
analysis['Portfolio Weight']= 1/len(tickers)

#Annualized Volatility (trailing 3 months), iloc[-1] slices the last row/ current data from each column.
analysis['Annualized Volatility'] = volatility_three.iloc[-1]

#Betas againts SPY,IWM and DIA
analysis = analysis.merge(df, left_index=True, right_on='Asset', how='left').set_index('Asset')
analysis = analysis.merge(df2, left_index=True, right_on='Asset', how='left').set_index('Asset')
analysis = analysis.merge(df3, left_index=True, right_on='Asset', how='left').set_index('Asset')

#Avg Weekly Drawdown
analysis['Avg Weekly Drawdown']= avg_drawdown

#Max Weekly Drawdown
analysis['Max Weekly Drawdown']= max_drawdown_values

#Total Returns
analysis['Total Returns']= total_return

#Annualized Total Return
analysis['Annualized Total Return']= annual_return
print('Table 1: ')
analysis

Table 1: 


,Portfolio Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Avg Weekly Drawdown,Max Weekly Drawdown,Total Returns,Annualized Total Return
Asset,,,,,,,,,
NVDA,0.142857,23.092307,2.06,0.50,1.23,-0.9132,-1.9092,17822.21,166.13
AMD,0.142857,25.423938,1.94,0.59,1.20,-0.9432,-2.7306,4710.35,132.98
GME,0.142857,41.335199,1.27,1.19,1.32,-0.9907,-2.0075,112.61,60.52
DIS,0.142857,15.395827,0.97,0.59,1.22,-0.9794,-2.1911,52.01,48.74
PFE,0.142857,13.607656,0.43,0.23,0.62,-0.8989,-2.5212,29.81,40.89
XOM,0.142857,9.681492,0.52,0.45,0.97,-0.9751,-2.3832,68.31,52.78
TGT,0.142857,10.330507,0.85,0.61,1.23,-2.9880,-63.5840,224.73,71.93
